In [1]:
""" fa file format

>target-long, T=0.07900000000000001, seed=1, num_res=394, num_ligand_res=124, use_ligand_context=True, ligand_cutoff_distance=8.0, batch_size=3, number_of_batches=10, model_path=./model_params/ligandmpnn_v_32_030_25.pt
GGGGFLVELLKELAEMAKVHVEALKIIAKIAANGGGGFLVELLKELAEMAKVQVEALKIIAKIAANGGGGFIVEILKELKEMAKVHVEQLKVLKKIAANGGGGDIVEILKELKELAKVQVEQLKVIKKIAKNGGGGFIVEILKELNKMAKEHVEALKIIKEIAAAGGGGDIVEILEKLNEMAKEQVEQLKVIKQIAKN:GGFLVELLKELAEMAKVHVEALKIIAKIAANGGGGFLVELLKELAEMAKVHVEALKIIAKIAANGGGGFIVEILKELKEMAKVQVEQLKVLKKIAANGGGGDIVEILKELKELAKVHVEQLKVIKKIAKNGGGGFIVEILKELNKMAKEQVEALKIIKEIAAAGGGGDIVEILEKLNEMAKEHVEQLKVIKQIAKN
>target-long, id=1, T=0.07900000000000001, seed=1, overall_confidence=0.4479, ligand_confidence=0.4828, seq_rec=0.4594
GGGALLVDLMKEMNEHAKEHVKILKILAEMVRAGGGDDIVEIFKELVEMAKIHVEALKVLREILEAGGVEQIVDVWKELNEHAKNHVKIWEIFAEMIEAGGGEDLVEIFKELNKMMKIHVEIFKVLREIFRAVGGEGIVARLKEWNKIGKEHVKAWKEVAEKFKKTGGEGLVDYLKELNKQGKKIVEELKKEAEKAAK:SSWLPEIAKLEKERKKIHKEAMKILKEMLDAGGKEDEPEVKKELKKMAEIHKEALKVLAEMLAAGGLDLAPAVWKELNELGKNHVKMLKIFAEMVRAGGGDDIVEIFKELNKMGKIHVEILKVLAEMLRAGGGDLLPAIFKEMNEIAKEHVKALKILAEMVRAGGGDDIVEIFKELNEMGKKHVEFLKVVAEIVSL
>target-long, id=2, T=0.07900000000000001, seed=1, overall_confidence=0.4525, ligand_confidence=0.4917, seq_rec=0.4746

"""

filename = "batch (13).fa"
sequences = {}
with open(filename, 'r') as file:
    lines = file.readlines()

current_sequence = ""
current_confidence = 0.0

for line in lines:
    if line.startswith(">"):
        if current_sequence:
            sequences[current_sequence] = current_confidence
        current_sequence = ""
        parts = line.split(", ")
        for part in parts:
            if part.startswith("ligand_confidence="):
                current_confidence = float(part.split("=")[1])
    else:
        current_sequence += line.strip()

if current_sequence:
    sequences[current_sequence] = current_confidence

unique_sequences = sorted(sequences.items(), key=lambda x: x[1], reverse=True)
top_5_sequences = unique_sequences[:5]
for top_sequence in top_5_sequences:
    parts = top_sequence[0].split(":")
    print(top_sequence[1])
    for part in parts:
        print(part)
    print()
bottom_5_sequences = unique_sequences[-5:]
for bottom_sequence in bottom_5_sequences:
    parts = bottom_sequence[0].split(":")
    print(bottom_sequence[1])
    for part in parts:
        print(part)
    print()


 

0.6456
KKEGEEKAKKKLEKLGKKGEEAGLKILKKFIKLLK
EEELKKELKKELAKIGKEGEKTFKEAIEEIKKLMG
GGEEIAKGLEKLMKKGIAGMKAGGKGLLAFFKLLK
GLKEIIKFLKVFGKNGIEGMKAFMEAFKAIMKLMG
GYEEIVKGLEKLMKKGIAGMKAGGKGLLAFFKLLK
GLKEIIKFLKVFGKNGIEGMKAFMEAFKAIMKLMG
GYEAIVKGLIKLMKKGIAGAKAGGKGLLEFFKLLK
GLEEMIKFLKVFGKGAIAGAKAFMKAFKEIMKLFG
GYEAIVKGLIKLMKKGIAGAKAGGKGLKEFFKLLK
GAEEMIKFLIVFGKGAIAGAKAFMKGFKEIMKLFG
GVEAIVKGLIKLMKKGIAGAKAGGKGLKEFFKLLK
GAEEMIKFLIVFGKGAIAGAKAFLKGFKEIMKLLG
GVEEIVDKILEKIKKAIEGAKKGVKKEIEKLKKKK
GAEKMIIFLIIKGIKAIIGAKKKLEEEKKKLKEEG

0.64
ELEALAALLKKIAAGAKAGHAIAKKLLAVILKVVK
SEEGLKKLKEVLKKLVKEGHAIFAATLAALAALAG
ELEALGKGLKKIGKGAIAGLKIWAEAFKVILKAVK
GGEGLVEILKELAKAVKEGLKIFGATLLAIAKLAG
ELEELSKGLKKIGKGAIAGLKIWAEAFKVILAAVK
GGEGLVEILKELAKAVKEGLKIFGATLIAIAKLIG
ELKEFSKGLKKIGKGAIAGLKIWAEAFKVILAAVK
GGEGLVKILKELAEKVKEGLKIFGAGLIAIAKLIG
ELKEFSKGLEKIGKGAIAGLKIWAEAFKVILEAVK
GGEGLVKLLKVLKEKEKEGLKIFGAGLIAIAKLIG
ELKEFGKGLEKIGKGAIAGVKIWAEAFKVILEAVK
GGEGLVKLLKVLKEKEIEGLKIFGAGLIAILKLIG
ELKKFGEWLEKIMEKAIEKAKKAAEKLKKVLEKIK
GGEGLVELGKELKKK

In [2]:
!pip install py3dmol biopython

In [3]:
import Bio.PDB as pdb
import numpy as np
from sklearn.decomposition import PCA
import py3Dmol
import numpy as np
from scipy.sparse.csgraph import shortest_path 
from scipy.sparse import csr_matrix
import numpy as np
import json
from scipy.spatial.distance import cdist
from matplotlib import pyplot as plt

def load_structure(file_path):
    """Load PDB or CIF file and return structure object"""
    parser = pdb.PDBParser(QUIET=True) if file_path.endswith('.pdb') else pdb.MMCIFParser(QUIET=True)
    structure = parser.get_structure('protein', file_path)
    return structure

def get_nanowire_axis(structure):
    """Calculate primary axis of the nanowire using PCA"""
    # Get all atom coordinates
    coords = []
    for atom in structure.get_atoms():
        coords.append(atom.get_coord())
    coords = np.array(coords)
    
    # Perform PCA to find primary axis
    pca = PCA(n_components=3)
    pca.fit(coords)
    primary_axis = pca.components_[0]  # First principal component
    return primary_axis

def find_heme_chains(structure):
    """Find all chains containing HEM or HEC residues"""
    heme_chains = []
    for chain in structure.get_chains():
        for residue in chain:
            if residue.get_resname() in ['HEM', 'HEC']:
                heme_chains.append(chain.id)
                break
    return list(set(heme_chains))

def get_heme_plane(residue):
    """Calculate plane for a heme group using major carbons"""
    # Get Fe atom
    fe_atom = None
    plane_atoms = []
    
    for atom in residue:
        if atom.name == 'FE':
            fe_atom = atom
        # Select carbon atoms for plane fitting (excluding carboxylic acids)
        elif atom.name.startswith('C') and not any(x in atom.name for x in ['A', 'D']):
            plane_atoms.append(atom)
    
    # Fit plane to atoms
    coords = np.array([atom.get_coord() for atom in plane_atoms])
    centroid = np.mean(coords, axis=0)
    
    # Calculate plane normal using SVD
    u, s, vh = np.linalg.svd(coords - centroid)
    normal = vh[2]
    return {'position':fe_atom.get_coord(), 'normal': normal,'centroid': centroid}



def create_rotation_matrix(primary_axis, average_normal):
    """Create rotation matrix to align primary_axis with x-axis and project average_normal onto y-z plane"""
    # First, create x-axis alignment
    x_axis = np.array([1, 0, 0])
    rotation_axis1 = np.cross(primary_axis, x_axis)
    rotation_axis1 /= np.linalg.norm(rotation_axis1)
    angle1 = np.arccos(np.dot(primary_axis, x_axis))
    
    # Create rotation matrix for primary axis alignment
    K1 = np.array([
        [0, -rotation_axis1[2], rotation_axis1[1]],
        [rotation_axis1[2], 0, -rotation_axis1[0]],
        [-rotation_axis1[1], rotation_axis1[0], 0]
    ])
    R1 = np.eye(3) + np.sin(angle1) * K1 + (1 - np.cos(angle1)) * np.matmul(K1, K1)
    
    # Rotate average_normal by R1
    rotated_normal = np.dot(R1, average_normal)
    
    # Project rotated_normal onto y-z plane
    projected_normal = np.array([0, rotated_normal[1], rotated_normal[2]])
    projected_normal /= np.linalg.norm(projected_normal)
    
    # Create second rotation to align projected_normal with y-axis
    y_axis = np.array([0, 1, 0])
    angle2 = np.arccos(np.dot(projected_normal, y_axis))
    if projected_normal[2] < 0:
        angle2 = -angle2
    
    # Create rotation matrix for y-axis alignment (rotating around x-axis)
    R2 = np.array([
        [1, 0, 0],
        [0, np.cos(angle2), -np.sin(angle2)],
        [0, np.sin(angle2), np.cos(angle2)]
    ])
    
    # Complete rotation matrix
    R = np.dot(R2, R1)
    return R

def transform_coords(coords, rotation_matrix):
    """Apply rotation matrix to coordinates"""
    return np.dot(coords, rotation_matrix.T)

def transform_structure_and_hemes(structure, hemeInfos, primary_axis, average_normal):
    """Transform structure and heme information to new coordinate system"""
    # Create rotation matrix
    R = create_rotation_matrix(primary_axis, average_normal)
    
    # Transform atomic coordinates in structure
    for atom in structure.get_atoms():
        new_coord = transform_coords(atom.get_coord(), R)
        atom.set_coord(new_coord)
    
    # Transform heme information
    for heme in hemeInfos:
        # Transform Fe position
        heme['position'] = transform_coords(heme['position'], R)
        
        # Transform normal vector
        heme['normal'] = transform_coords(heme['normal'], R)
        
        # Transform centroid
        heme['centroid'] = transform_coords(heme['centroid'], R)
    
    # Sort hemes based on x-coordinate (horizontal axis)
    hemeInfos.sort(key=lambda x: x['position'][0])
    
    
    
    return structure, hemeInfos, R 


def get_vdw_radius():
    """Return van der Waals radii for relevant atoms (in Angstroms)"""
    return {
        'C': 1.70,
        'N': 1.55,
        'O': 1.52,
        'FE': 2.04,
    }

def get_porphyrin_ring_carbons(residue):
    """Get only the carbon atoms that form the porphyrin ring structure"""
    ring_carbons = []
    
    # Pattern for porphyrin ring carbons (typically labeled CA, CB, etc.)
    ring_carbon_patterns = ['CA', 'CB', 'CG', 'CD']
    
    for atom in residue:
        # Check if atom is a carbon and matches porphyrin ring pattern
        if (atom.name.startswith('C') and 
            any(pattern in atom.name for pattern in ring_carbon_patterns) and
            not any(x in atom.name for x in ['CAA', 'CAB', 'CAC', 'CAD',  # Exclude propionate carbons
                                           'CBA', 'CBB', 'CBC', 'CBD'])):
            ring_carbons.append(atom)
    
    return ring_carbons

def calculate_edge_distances(hemeInfos):
    """Calculate edge-to-edge distances between adjacent heme pairs using only porphyrin ring carbons"""
    distances = np.zeros((len(hemeInfos), len(hemeInfos)))
     
    overlaps=0
    for i in range(len(hemeInfos) ):
        for j in range(i+1,len(hemeInfos)):
            heme1 = hemeInfos[i]
            heme2 = hemeInfos[j]
            
            # Get only porphyrin ring carbons
            carbons1 = get_porphyrin_ring_carbons(heme1['residue'])
            carbons2 = get_porphyrin_ring_carbons(heme2['residue'])
            
            # Calculate all pairwise distances between ring carbons
            coords1 = np.array([atom.get_coord() for atom in carbons1])
            coords2 = np.array([atom.get_coord() for atom in carbons2])
            
            # Calculate minimum distance accounting for van der Waals radii
            dist_matrix = cdist(coords1, coords2)
            min_idx = np.unravel_index(np.argmin(dist_matrix), dist_matrix.shape)
            
            # Carbon vdW radius is 1.70 Å
            edge_distance = dist_matrix[min_idx] - (2 * 1.70)  # Subtract 2x carbon vdW radius
            if edge_distance < 0:
                overlaps+=edge_distance
                edge_distance = 0
                
            distances[i,j]= edge_distance
            
    return distances, overlaps 

def calculate_pi_overlap(hemeInfos,distances):
    # Find neighboring hemes based on edge-to-edge distances
    neighbor_pairs = []
    for i in range(len(hemeInfos)):
        for j in range(i + 1, len(hemeInfos)):
            if distances[i, j] < 8:
                neighbor_pairs.append((i, j))


    # Filter overlaps to include only neighboring hemes
    filtered_overlaps = np.zeros((len(hemeInfos), len(hemeInfos)))
    for i, j in neighbor_pairs:
        heme1 = hemeInfos[i]
        heme2 = hemeInfos[j]
        
        # Get porphyrin plane normals
        normal1 = heme1['normal']
        normal2 = heme2['normal']
        
        # Calculate angle between planes
        angle = np.arccos(np.abs(np.dot(normal1, normal2))) * 180 / np.pi
        
        # Calculate center-to-center vector
        center_vector = heme2['centroid'] - heme1['centroid']
            
        # Calculate overlap area (approximate as intersection of projected circles)
        radius = 4.8  # Approximate radius of porphyrin ring
        d = np.linalg.norm(center_vector)
        
        if d > 2 * radius:
            overlap_area =  (np.pi * 1.70**2) # Add area of carbon atom for handling hemes that are oriented edge-to-edge
        else:
            # Area of intersection of two circles
            a = np.arccos(d/(2*radius))
            overlap_area = 2 * (radius**2 * a - d/2 * radius * np.sin(a))
            # Adjust for angle between planes
            overlap_area *= np.cos(angle * np.pi / 180)
        
        filtered_overlaps[i, j]=  overlap_area + (np.pi * 1.70**2) # Add area of carbon atom for handling hemes that are oriented edge-to-edge

    return filtered_overlaps

def estimate_electron_transfer_rate(distances, overlaps, temperature=298):
    """Estimate electron transfer rates using modified Marcus theory"""
    rates = []
    kb = 8.617333262e-5  # Boltzmann constant in eV/K
    
    # Parameters for electron transfer rate calculation
    beta = 1.4  # Distance decay factor (Å⁻¹)
    V0 = 200  # Maximum coupling at contact (cm⁻¹)
    lambda_ = 0.7  # Reorganization energy (eV)
    dG = 0  # Assume no driving force between identical hemes
    
    for d, o in zip(distances, overlaps):
        # Calculate electronic coupling
        V = V0 * np.exp(-beta * d['edge_distance']) * (o['overlap_area'] / (np.pi * 4.8**2))
        
        # Calculate Marcus rate
        kET = (2 * np.pi / 6.582e-16) * (V/1.23984e-4)**2 * \
              np.exp(-(lambda_ + dG)**2 / (4 * lambda_ * kb * temperature))
        
        rates.append({
            'heme_pair': d['heme_pair'],
            'rate': kET,
            'coupling': V
        })
    
    return rates


 

def find_longest_short_jump_path(distances, max_jump=6.0):
    # Create adjacency matrix for valid jumps
    n = len(distances)
    adj_matrix = np.zeros_like(distances)
    for i in range(n):
        for j in range(n):
            if 0 < distances[i,j] <= max_jump:
                # Weight is inverse of distance to favor shorter jumps
                adj_matrix[i,j] = 1.0/distances[i,j]
                adj_matrix[j,i] = 1.0/distances[i,j]
    
    # Find all possible paths between every pair of nodes
    sparse_adj = csr_matrix(adj_matrix)
    dist_matrix, predecessors = shortest_path(sparse_adj, 
                                            return_predecessors=True,
                                            directed=False)
    
    # Find the longest valid path
    best_path = []
    max_length = 0
    
    for start in range(n):
        for end in range(start+1, n):
            if dist_matrix[start,end] != np.inf:
                # Reconstruct path
                path = []
                curr = end
                while curr != start:
                    path.append(curr)
                    curr = predecessors[start,curr]
                path.append(start)
                path.reverse()
                
                # Calculate actual physical length
                path_length = sum(distances[path[i], path[i+1]] 
                                for i in range(len(path)-1))
                
                if len(path) > len(best_path):
                    best_path = path
                    max_length = path_length
    
    return best_path, max_length

def calculate_pathway_conductance(path, distances, overlaps):
    # Calculate conductance using modified Landauer formula
    # Assuming coherent tunneling through the wire
    
    # Constants
    h = 6.62607015e-34  # Planck's constant
    e = 1.60217663e-19  # Elementary charge
    
    # Calculate total resistance along path
    total_resistance = 0
    for i in range(len(path)-1):
        heme1, heme2 = path[i], path[i+1]
        
        # Get distance and overlap for this pair
        distance = distances[heme1,heme2]
        overlap = overlaps[heme1,heme2]
        
        # Calculate local transmission coefficient
        # Using simplified model where transmission decays exponentially with distance
        # and scales linearly with overlap
        beta = 1.4  # Å⁻¹ (typical for proteins)
        transmission = np.exp(-beta * distance) * (overlap / (np.pi * 4.8**2))
        
        # Add resistance (proportional to 1/transmission)
        if transmission > 0:
            total_resistance += 1/transmission
    
    # Calculate total conductance (in units of quantum conductance G0)
    G0 = 2 * e**2 / h  # Quantum of conductance
    conductance = G0 / (total_resistance+.00000000001)
    
    return conductance
 
def visualize_structure(structure, primary_axis, hemeInfos, best_path):
    """Create py3Dmol visualization with axis, heme planes, and selected path"""
    viewer = py3Dmol.view(width=800, height=600)
    
    # Convert structure to PDB format for py3Dmol
    io = pdb.PDBIO()
    io.set_structure(structure)
    io.save('temp.pdb')
    
    with open('temp.pdb') as f:
        pdb_data = f.read()
    
    # Add protein structure
    viewer.addModel(pdb_data, "pdb")
    viewer.setStyle({'cartoon': {'color': 'white'}})
    viewer.setStyle({'hetflag': True}, {'stick': {'colorscheme': 'yellowCarbon'}})
    
    # Add primary axis arrow
    center = (np.mean([atom.get_coord() for atom in structure.get_atoms()], axis=0)).astype(float).tolist()
    end = (center + primary_axis * 100).astype(float).tolist()  # Scale arrow length
    viewer.addArrow({
        'start': {'x': center[0], 'y': center[1], 'z': center[2]},
        'end': {'x': end[0], 'y': end[1], 'z': end[2]},
        'radius': 0.3,
        'color': 'blue'
    })
    
    axis_length = 10  # Length of axis arrows
    for i, heme in enumerate(hemeInfos):
        pos = heme['position']
        normal = heme['normal'] * axis_length
        end = pos + normal
        
        # Draw arrow for heme normal
        viewer.addArrow({
            'start': {'x': float(pos[0]), 'y': float(pos[1]), 'z': float(pos[2])},
            'end': {'x': float(end[0]), 'y': float(end[1]), 'z': float(end[2])},
            'radius': 0.3,
            'color': 'green'
        })
    
    # Draw curved line for the selected path
    path_coords = [hemeInfos[i]['position'] for i in best_path]
    for i in range(len(path_coords) - 1):
        viewer.addCylinder({
            'start': {'x': float(path_coords[i][0]), 'y': float(path_coords[i][1]), 'z': float(path_coords[i][2])},
            'end': {'x': float(path_coords[i+1][0]), 'y': float(path_coords[i+1][1]), 'z': float(path_coords[i+1][2])},
            'radius': 0.2,
            'color': 'red'
        })
    
    viewer.zoomTo()
    return viewer   

def CheckEnds(hemeInfos,best_path,structure):
    # Extract x-coordinates of the start and end hemes in the best_path
    start_heme_x = hemeInfos[best_path[0]]['position'][0]
    end_heme_x = hemeInfos[best_path[-1]]['position'][0]

    # Determine the extent of the non-heme chains along the x-axis
    # Exclude heme atoms when calculating x-coordinates
    heme_residues = {heme['residue'] for heme in hemeInfos}
    all_x_coords = [atom.get_coord()[0] for atom in structure.get_atoms() if atom.get_parent() not in heme_residues ] 
    min_x = min(all_x_coords)
    max_x = max(all_x_coords)

    # Calculate the distance from the ends of the protein to the start and end hemes
    distance_start_heme_to_protein_start = min(abs(start_heme_x - min_x), abs(start_heme_x - max_x))
    distance_end_heme_to_protein_end = min(abs(end_heme_x - min_x), abs(end_heme_x - max_x))

    # Width of an alpha helix (approximately 10 Å)
    alpha_helix_width = 10.0

    return np.mean([distance_start_heme_to_protein_start/alpha_helix_width, distance_end_heme_to_protein_end/alpha_helix_width])    

In [5]:
def loadAndRotate(filename):
    # Load structure
    structure = load_structure(filename)

    # Get primary axis
    primary_axis = get_nanowire_axis(structure)
    

    # Find heme chains
    heme_chains = find_heme_chains(structure)
   

    # Analyze each heme
    hemeInfos=[]
    for chain in structure.get_chains():
        if chain.id in heme_chains:
            for residue in chain:
                if residue.get_resname() in ['HEM', 'HEC']:
                    hemeInfo = get_heme_plane(residue)
                    hemeInfo['chainID'] = chain.id
                    hemeInfo['chain'] = chain
                    hemeInfo['residue'] = residue
                    hemeInfos.append(hemeInfo)
                    break
                
    # Determine the average normal direction
    average_normal = np.mean([heme['normal'] for heme in hemeInfos], axis=0)
    average_normal /= np.linalg.norm(average_normal)  # Normalize the average normal

    # Flip normals that are antiparallel to the average normal
    for heme in hemeInfos:
        if np.dot(heme['normal'], average_normal) < 0:
            heme['normal'] = -heme['normal']
            
    structure, hemeInfos,_=transform_structure_and_hemes(structure, hemeInfos, primary_axis, average_normal)        
            
    primary_axis = get_nanowire_axis(structure)    
    return structure,primary_axis,hemeInfos

def AnalyzeProtein(filename):
    """Main function to analyze heme nanowire"""
    structure,primary_axis,hemeInfos =  loadAndRotate(filename)
            
    distances,clashes = calculate_edge_distances(hemeInfos)
    overlaps = calculate_pi_overlap(hemeInfos,distances)

    # Find best path
    best_path, path_length = find_longest_short_jump_path(distances)
    
    if len(best_path) < 2:
        return None

    # Calculate conductance along path
    conductance = calculate_pathway_conductance(best_path, distances, overlaps)
    endRating = CheckEnds(hemeInfos,best_path,structure)
    
    info = {
        'file': filename,
        'path': best_path,
        'path_length': path_length,
        'conductance': conductance,
        'hop_distances': [distances[best_path[i], best_path[i+1]] 
                            for i in range(len(best_path)-1)],
        'path_overlaps': [overlaps[best_path[i], best_path[i+1]] 
                            for i in range(len(best_path)-1)],
        'endRating': endRating,
       
        'primary_axis': primary_axis,
        'hemeInfos': [ {'position':x['position'],'normal':x['normal']} for x in  hemeInfos ],
        'hemeClashes': clashes
    }    
    
    with open(filename.replace('model','summary_confidences').replace(".cif",'.json'), 'r') as file:
        summary = json.load(file)
        info[ 'ipTM']=(summary['iptm'])
        info['pTM']=(summary['ptm'])
        info['hasClash']= summary['has_clash']
        info['fraction_disordered']= summary['fraction_disordered']
        
        
    with open(filename.replace('model','full_data').replace(".cif",'.json') , 'r') as file:
        summary = json.load(file)
        info['pae']=np.max (summary['pae'])
        info['contact_probs']=np.mean(summary['contact_probs'])

    return info      
        
        



In [ ]:
import os
import glob

fold_Folder =r'D:\Downloads\ProteinNanowires\folds'
cif_files = []
proteinDetails = []
# Extract each zip file into a temp folder and get a list of the .cif files
 
cif_files = glob.glob(os.path.join(fold_Folder, '**', '*.cif'), recursive=True)
for cFile in cif_files:
    try:
        proteinRating = AnalyzeProtein(cFile)
        if proteinRating is None:
            continue
        if len(proteinRating['path'])<10:
            continue
        
        print(f'{len(proteinRating["path"])} - {proteinRating["conductance"]} - {proteinRating["ipTM"]} : {os.path.basename( cFile)} ')
        proteinDetails.append(proteinRating)
    except:
        pass
                
            
              


In [ ]:
#sort by ipTM
proteinDetails.sort(key=lambda x: x['ipTM'], reverse=True)
safe = [x for x in proteinDetails if x['hemeClashes']  >=0]

candidates = safe[0:np.min([10,len(safe)])]
#sort by conductance
candidates.sort(key=lambda x: x['conductance'], reverse=True)

#load the highest ipTM and plot the structure
for i in range( 5):
    highest_ipTM = candidates[i]
     
    structure,primary_axis,hemeInfos =  loadAndRotate(highest_ipTM['file'])
    
    # Create visualization
    viewer = visualize_structure(structure,  primary_axis,hemeInfos, highest_ipTM['path'])
    #add conductance to the graph
    print(f'Conductance: {highest_ipTM["conductance"]*1e9:.2f} nS^-1')
    print(f'heme clashes: { highest_ipTM['hasClash']} {highest_ipTM["hemeClashes"]}')
    print(f'Disordered: {highest_ipTM["fraction_disordered"]}')
    print(f'pae { highest_ipTM["pae"]}')
    viewer.zoomTo()    
    viewer.show()



In [12]:
best_path

[]

In [19]:

structure,primary_axis,hemeInfos =  loadAndRotate('target-complex.pdb')
            
distances,clashes = calculate_edge_distances(hemeInfos)
overlaps = calculate_pi_overlap(hemeInfos,distances)

# Find best path
best_path, path_length = find_longest_short_jump_path(distances)
 
# Calculate conductance along path
 

# Rotate every other heme chain by 180 degrees
for i, heme in enumerate(hemeInfos):
    if i % 2 == 1:
        # Create a 180-degree rotation matrix around the heme's normal axis
        angle = np.pi  # 180 degrees in radians
        axis = heme['normal'] / np.linalg.norm(heme['normal'])
        K = np.array([
            [0, -axis[2], axis[1]],
            [axis[2], 0, -axis[0]],
            [-axis[1], axis[0], 0]
        ])
        R = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * np.matmul(K, K)
        
        # Rotate all atoms in the heme residue around its centroid
        center = heme['position']
        for atom in heme['residue']:
            coord = atom.get_coord() - center
            new_coord = np.dot(R, coord) + center
            atom.set_coord(new_coord)


startHeme = hemeInfos[0]
endHeme = hemeInfos[-1]
# Calculate the total length between the start and end hemes
total_length = np.linalg.norm(endHeme['position'] - startHeme['position'])

# Calculate the number of segments (number of hemes - 1)
num_segments = len(hemeInfos) - 1

# Calculate the distance between each heme
segment_length = total_length / num_segments

# Calculate the direction vector from start to end heme
direction_vector = (endHeme['position'] - startHeme['position']) / total_length

# Place each heme equidistantly along the direction vector
# for i  in range(1,len(hemeInfos)-1):
#     heme=hemeInfos[i]
#     heme['position'] = startHeme['position'] + direction_vector * segment_length * i
#     # Update the coordinates of all atoms in the heme residue
#     for atom in heme['residue']:
#         coord = atom.get_coord() - heme['centroid']
#         new_coord = coord + heme['position']
#         atom.set_coord(new_coord)

# Save the modified structure to a new PDB file
io = pdb.PDBIO()
io.set_structure(structure)

io.save('target2.pdb')

In [16]:
endHeme 

{'position': array([56.16799927, -8.74728707,  6.41584616]),
 'normal': array([-0.49918163, -0.66171556,  0.55941955]),
 'centroid': array([56.10543442, -6.92218581,  8.67714519]),
 'chainID': 'T',
 'chain': <Chain id=T>,
 'residue': <Residue HEM het=H_HEM resseq=1 icode= >}

In [2]:
'DIVEILEKLNEMAKEQVEQLKVIKQIAKN'[::-1]

'NKAIQKIVKLQEVQEKAMENLKELIEVID'